load model

In [1]:
import pandas as pd
import joblib
import xgboost as xgb

# Load the models from a file
fare_model = joblib.load('best_fare_model.pkl')
distance_model = xgb.Booster()
distance_model.load_model('distance_model.model')

# Assume we have some new data             (add data here)
new_data = pd.DataFrame({
    'pickup_hour': [10, 15, 20],
    'PULocationID': [161, 237, 43],
    'DOLocationID': [234, 161, 68]
})                                      #(change the new_data)

# new_data = pd.DataFrame({
#     'pickup_hour': [13],  # 1 PM
#     'PULocationID': [132],  # Some location ID
#     'DOLocationID': [234]  # Some other location ID
# })





# Make predictions
fare_predictions = fare_model.predict(new_data)
distance_predictions = distance_model.predict(xgb.DMatrix(new_data))

# Print predictions
print('Fare predictions:', fare_predictions)
print('Distance predictions:', distance_predictions)



Fare predictions: [10.84505174 10.20286437 15.118387  ]
Distance predictions: [1.4689379 1.3689122 2.5435712]


In [2]:
import sklearn
print(sklearn.__version__)

1.3.0


model test

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


normal test

In [16]:
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv('yellow_manhattan.csv')

# Preprocess the data
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Define features and target
features = df[['pickup_hour', 'PULocationID', 'DOLocationID']]
target_fare = df['real_total_amount']
target_distance = df['trip_distance']

# Split the data into training and test sets (90% training, 10% test)
features_train, features_test, target_fare_train, target_fare_test = train_test_split(features, target_fare, test_size=0.1, random_state=42)
_, _, target_distance_train, target_distance_test = train_test_split(features, target_distance, test_size=0.1, random_state=42)

# Load the models
fare_model = joblib.load('fare_model.pkl')
distance_model = joblib.load('distance_model.pkl')

# Make predictions
fare_predictions = fare_model.predict(features_test)
distance_predictions = distance_model.predict(features_test)

# Calculate MSE
fare_mse = mean_squared_error(target_fare_test, fare_predictions)
distance_mse = mean_squared_error(target_distance_test, distance_predictions)

print('Fare MSE:', fare_mse)
print('Distance MSE:', distance_mse)


Fare MSE: 16786.516379147335
Distance MSE: 2120.552255009547


# taxi fare

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

# Load the data
df = pd.read_csv('yellow_taxi_data_Q2.csv')

# Preprocess the data
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Define features and target
features = df[['pickup_hour', 'PULocationID', 'DOLocationID']]
target_fare = df['real_total_amount']

# Split the data into training and test sets
features_train, features_test, target_fare_train, target_fare_test = train_test_split(features, target_fare, test_size=0.2, random_state=42)

# Define the model
model = RandomForestRegressor(random_state=42)

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Set up the randomized search
# randomized_cv = RandomizedSearchCV(estimator=model, param_distributions=hyperparameter_grid, n_iter=10, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2, random_state=42)
# Set up the randomized search with n_jobs=1
# randomized_cv = RandomizedSearchCV(estimator=model, param_distributions=hyperparameter_grid, n_iter=10, cv=5, scoring='neg_mean_squared_error', n_jobs=1, verbose=2, random_state=42)

# Set up the randomized search with n_jobs=1
randomized_cv = RandomizedSearchCV(estimator=model, param_distributions=hyperparameter_grid, n_iter=4, cv=5, scoring='neg_mean_squared_error', n_jobs=1, verbose=2, random_state=42)


# Fit the randomized search
randomized_cv.fit(features_train, target_fare_train)

# Get the best model
best_model = randomized_cv.best_estimator_

# Make predictions
fare_predictions = best_model.predict(features_test)

# Calculate MSE
fare_mse = mean_squared_error(target_fare_test, fare_predictions)

print('Fare MSE:', fare_mse)

# Save the model to a file
joblib.dump(best_model, 'best_fare_model.pkl')



Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=  50.5s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=  54.1s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=  53.8s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=  53.2s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=  52.9s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 4.1min
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 5.1min
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 4.6min
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time= 4.1min
[CV] END max_depth=None

['best_fare_model.pkl']

xgboost

In [9]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('yellow_manhattan.csv')

# Preprocess the data
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Define features and target
features = df[['pickup_hour', 'PULocationID', 'DOLocationID']]
target_fare = df['real_total_amount']
target_distance = df['trip_distance']

# Split the data into training and test sets
features_train, features_test, target_fare_train, target_fare_test = train_test_split(features, target_fare, test_size=0.2, random_state=42)
_, _, target_distance_train, target_distance_test = train_test_split(features, target_distance, test_size=0.2, random_state=42)

# Create DMatrix for train and test
dtrain_fare = xgb.DMatrix(features_train, label=target_fare_train)
dtrain_distance = xgb.DMatrix(features_train, label=target_distance_train)
dtest = xgb.DMatrix(features_test)

# Define parameters
param = {'max_depth': 10, 'eta': 0.3, 'objective': 'reg:squarederror'}

# Train the models
num_round = 10
fare_model = xgb.train(param, dtrain_fare, num_round)
distance_model = xgb.train(param, dtrain_distance, num_round)

# Save the models
# fare_model.save_model('fare_model.model')
distance_model.save_model('distance_model.model')


xgboost  test

In [10]:
# Load the models
loaded_fare_model = xgb.Booster()
loaded_fare_model.load_model('fare_model.model')

loaded_distance_model = xgb.Booster()
loaded_distance_model.load_model('distance_model.model')

# Make predictions
fare_predictions = loaded_fare_model.predict(dtest)
distance_predictions = loaded_distance_model.predict(dtest)

# Calculate MSE
fare_mse = mean_squared_error(target_fare_test, fare_predictions)
distance_mse = mean_squared_error(target_distance_test, distance_predictions)

print('Fare MSE:', fare_mse)
print('Distance MSE:', distance_mse)


Fare MSE: 14098.187742214215
Distance MSE: 1248.6357850431907
